# To-Do
* only get negative examples completely in bangladesh [done]
* read composite images directly from drive
* only need to snip the cols in the last row and the rows in the last col
* create github [done]
* save data to .data [done]
* don't upload .data to github [done]

In [1]:
import os
import json
import requests
import time
from requests.auth import HTTPBasicAuth
import pandas as pd
import datetime
import rasterio
from matplotlib import pyplot as plt
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from os import listdir
from os.path import isfile, join

In [96]:
!pip install oauth2client

     |████████████████████████████████| 98 kB 4.5 MB/s eta 0:00:01


In [101]:
from apiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools
obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('data/client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=741294494890-5sidvm6q429r14a0323gi30km3is2f1p.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&access_type=offline&response_type=code

Enter verification code: 4/1AY0e-g6hvEwTxw-phAiGdh58P79GOkIItuaDlVuhX3Yj7ZRGtdcY0FdILMw
Authentication successful.


In [2]:
datasets = []

# read datasets from folder
folder_name = "data/b_composites/"
dataset_files = [f for f in listdir(folder_name) if isfile(join(folder_name, f))]
dataset_files.sort()
print("Reading files:", dataset_files)

for file in dataset_files:
    datasets += [rasterio.open(folder_name + file)]
    
# calculate image grid
first_x_coord = dataset_files[0].split(".")[0].split("-")[1]
first_y_coord = dataset_files[0].split(".")[0].split("-")[2]
num_image_cols = len([x for x in dataset_files if x.split(".")[0].split("-")[1] == first_x_coord])
num_image_rows = len([x for x in dataset_files if x.split(".")[0].split("-")[2] == first_y_coord])
print("Number of image grid columns:", num_image_cols)
print("Number of image grid rows:", num_image_rows)

Reading files: ['bangladesh_all_bands_final-0000000000-0000000000.tif', 'bangladesh_all_bands_final-0000000000-0000006656.tif', 'bangladesh_all_bands_final-0000006656-0000000000.tif', 'bangladesh_all_bands_final-0000006656-0000006656.tif']
number of image grid columns: 2
number of image grid rows: 2


In [3]:
# set params
tile_height, tile_length = (64, 64)
bands_to_read = None # ['B4', 'B3', 'B2'] # set to None to read all bands
examples_per_save_file = 1000
save_path = 'data/tiles/'

# resources
kilns = pd.read_csv("data/bangladesh_kilns.csv")
all_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B8', 'B9', 'B10', 'B11', 'B12']
band_dict = dict(zip(all_bands, range(1, len(all_bands) + 1)))

print(kilns.head())

   index        lat        lon     x    y      prob prediction   shape
0      0  23.796287  90.316715  1043  518  0.564438     zigzag  zigzag
1      1  23.940202  90.043026    78  149  0.692853     zigzag  zigzag
2      2  25.021253  89.309124   636  327  0.499505        fck     fck
3      3  25.308184  89.550603    97   85  0.342437        fck  zigzag
4      4  22.561670  92.007151   145  175  0.922128     zigzag  zigzag


In [10]:
coords = []
with open("data/countries.geojson", "r") as countries_geojson:
    country_dict = json.load(countries_geojson)["features"]
for obj in country_dict:
    name = obj['properties']['ADMIN']
    if name == "Bangladesh":
        coords = obj['geometry']["coordinates"]
flat_coords = []
for sublist in coords:
    for coord in sublist:
        for c in coord:
            flat_coords.append(c)

bangladesh_geo = Polygon(flat_coords)

True


In [43]:
print(datasets[0])
print(datasets[0].bounds)

<open DatasetReader name='data/b_composites/bangladesh_all_bands_final-0000000000-0000000000.tif' mode='r'>
BoundingBox(left=88.02178244056496, bottom=26.02563108539713, right=88.61970109367492, top=26.623549738507084)


In [74]:
def get_bands_from_dataset(dataset, band_names=None):
    start_time = time.time()
    print("Reading " + ("all" if band_names is None else str(len(band_names))) 
          + " bands from dataset " + str(dataset.name))
    if band_names is None: # get all bands
        bands = dataset.read()
    else:
        bands = []
        for idx, band_name in enumerate(band_names):
            band_index = band_dict[band_name]
            bands += [dataset.read(band_index)]
        bands = np.array(bands)
    print("time to read bands: " + str(time.time() - start_time))
    return bands

def get_data_given_pixels(ds_bounds, bands, start_row, start_col, contains_kiln):
    num_bands, ds_height, ds_length = bands.shape
    tile_top = ds_bounds.top - (start_row / ds_height) * (ds_bounds.top - ds_bounds.bottom)
    tile_bottom = ds_bounds.top - ((start_row + tile_height) / ds_height) * (ds_bounds.top - ds_bounds.bottom)
    tile_left = ds_bounds.left + (start_col / ds_length) * (ds_bounds.right - ds.bounds.left)
    tile_right = ds_bounds.left + ((start_col + tile_length) / ds_length) * (ds_bounds.right - ds.bounds.left)
    tile_coordinates = [[tile_left, tile_top], [tile_right, tile_top], [tile_right, tile_bottom], [tile_left, tile_bottom]]
#     print("top", tile_top, "bottom", tile_bottom, "left", tile_left, "right", tile_right)
    
    if not contains_kiln:
        for point in [Point([c[0], c[1]]) for c in tile_coordinates]:
            if not bangladesh_geo.contains(point):
                return None
    return bands[:, start_row : start_row + tile_height, start_col : start_col + tile_length]

# def add_example(ex_data, save_index, list_to_add, is_positive):
def add_example(ex_data, save_index, counter, examples, is_positive):
    examples[counter] = ex_data
    new_counter = counter + 1
    # save files if needed
    if new_counter == examples_per_save_file:
        filename = save_path + ("pos" if is_positive else "neg") + "_examples_" + str(save_index) + ".csv"
        print("Saving file", filename)
        np.savetxt(filename, examples.flatten(), delimiter=",")
        print("Finished saving file")
        save_index += 1
        new_counter = 0
    return new_counter, save_index

def get_kiln_tiles(bounds, num_rows, num_cols):
    kilns_in_image = kilns.loc[(kilns['lat'] >= bounds['bottom']) & (kilns['lat'] <= bounds['top']) 
        & (kilns['lon'] >= bounds['left']) & (kilns['lon'] <= bounds['right'])]
    
    tiles = set() # set of tuples of (tile_row, tile_col)
    for index, kiln in kilns_in_image.iterrows():
        lon_percent = 1 - (kiln['lon'] - bounds['left']) / (bounds['right'] - bounds['left'])
        row_index = int(num_rows * lon_percent)
        
        lat_percent = 1 - (kiln['lat'] - bounds['bottom']) / (bounds['top'] - bounds['bottom'])
        col_index = int(num_cols * lat_percent)
        tiles.add((row_index, col_index))
    return tiles
        

In [13]:
data_pixels = []
for ds in datasets:
    data_pixels += [get_bands_from_dataset(ds, band_names=bands_to_read)]
    
print("Data shape:", data_pixels[0].shape)

Reading all bands from dataset data/b_composites/bangladesh_all_bands_final-0000000000-0000000000.tif
time to read bands: 22.185380935668945
Reading all bands from dataset data/b_composites/bangladesh_all_bands_final-0000000000-0000006656.tif
time to read bands: 22.828599214553833
Reading all bands from dataset data/b_composites/bangladesh_all_bands_final-0000006656-0000000000.tif
time to read bands: 23.33644390106201
Reading all bands from dataset data/b_composites/bangladesh_all_bands_final-0000006656-0000006656.tif
time to read bands: 21.856069087982178
Data shape: (13, 6656, 6656)


In [113]:
pos_save_index, neg_save_index = 0, 0
pos_counter, neg_counter = 0, 0

pos_x_examples = np.zeros([examples_per_save_file, len(all_bands) if bands_to_read is None else len(bands_to_read), tile_height, tile_length])
neg_x_examples = np.zeros([examples_per_save_file, len(all_bands) if bands_to_read is None else len(bands_to_read), tile_height, tile_length])
print("Examples shape:", pos_x_examples.shape)

Examples shape: (1000, 13, 64, 64)


In [117]:
for ds_index, ds in enumerate(datasets[:1]):
    bands = data_pixels[ds_index]
    num_bands, ds_height, ds_length = bands.shape
    print("bands.shape", num_bands, ds_height, ds_length)
    num_rows = ds_height // tile_height
    num_cols = ds_length // tile_length
    print("total rows and cols", num_rows, num_cols)
    
    bounds = {
        "bottom": ds.bounds.bottom,
        "top": ds.bounds.top,
        "left": ds.bounds.left,
        "right": ds.bounds.right
    }
    
    row_px_excess, col_px_excess = None, None
    percent_row_excess, percent_col_excess = None, None
    
    if ds_index % num_image_cols == num_image_cols - 1: # rightmost column
        # calculate excess col pixels
        col_px_excess = ds_length % tile_length
        bounds["right"] -= col_px_excess / ds_length * (ds.bounds.right - ds.bounds.left)
    
    if ds_index // num_image_cols == num_image_rows - 1: # last row
        # calculate excess row pixels
        row_px_excess = ds_height % tile_height
        bounds["bottom"] += row_px_excess / ds_height * (ds.bounds.top - ds.bounds.bottom)
    
    kiln_tiles = get_kiln_tiles(bounds, num_rows, num_cols)
    print("Num tiles with kilns:", len(kiln_tiles))
    print(kiln_tiles)
    
    print("Start tiling dataset " + ds.name)
    start_time = time.time()
    for tile_idx_row in range(0, num_rows // 2):
        px_row = tile_idx_row * tile_height
        for tile_idx_col in range(0, num_cols // 2):
            px_col = tile_idx_col * tile_length
            if (tile_idx_row, tile_idx_col) in kiln_tiles:
                data = get_data_given_pixels(ds.bounds, bands, px_row, px_col, True)
                pos_counter, pos_save_index = add_example(data, pos_save_index, pos_counter, pos_x_examples, True)
            else:
                data = get_data_given_pixels(ds.bounds, bands, px_row, px_col, False)
                if data is not None:
                    neg_counter, neg_save_index = add_example(data, neg_save_index, neg_counter, neg_x_examples, False)
    print("Finished in", time.time() - start_time)

bands.shape 13 6656 6656
total rows and cols 104 104
Num tiles with kilns: 11
{(27, 98), (58, 91), (12, 71), (13, 71), (43, 78), (27, 97), (45, 67), (30, 99), (25, 98), (31, 99), (28, 95)}
Start tiling dataset data/b_composites/bangladesh_all_bands_final-0000000000-0000000000.tif
Finished in 1.1504392623901367


In [118]:
# print(neg_x_examples[10])
print(ds.bounds)
print(bounds)

BoundingBox(left=88.02178244056496, bottom=26.02563108539713, right=88.61970109367492, top=26.623549738507084)
{'bottom': 26.02563108539713, 'top': 26.623549738507084, 'left': 88.02178244056496, 'right': 88.61970109367492}


In [43]:
np.array(neg_x_list).shape
# np.array(pos_x_list).shape

(5000, 13, 64, 64)

In [65]:
cropped = np.array(bands)[:, :64, :64]
print(cropped.shape)

(3, 64, 64)


In [98]:
print(tile_bounds)

{"geometry": {"coordinates": [[[88.619701, 25.437328], [88.629316, 25.437328], [88.629316, 25.427712], [88.619701, 25.427712]]], "type": "Polygon"}, "properties": {}, "type": "Feature"}
